In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\vnathani\Desktop\Material_Request_Details_Analysis\grddyn.csv')

df = df[['Aircraft Reg #', 'Material Request #', 'MR Created Date', 'MR Need Date', 'MR Part #', 'Part Description', 'MR Qty.', 'Issued Qty', 'PO #', 'STI #', 'Vendor Name', 'Requested By', 'Task#']]

df.insert(df.columns.get_loc('Vendor Name') + 1, 'Tracking #', '')
df.insert(df.columns.get_loc('Tracking #') + 1, 'Current ETA', '')
df.insert(df.columns.get_loc('Current ETA') + 1, 'Notes:', '')
df.insert(df.columns.get_loc('Task#') + 1, 'Escalate', '')
df.insert(df.columns.get_loc('Escalate') + 1, 'Shipped', '')
df.insert(df.columns.get_loc('Shipped') + 1, 'Delivered', '')
df.insert(df.columns.get_loc('Delivered') + 1, 'Received', '')

df.to_csv('grddyn.csv', index=False)

filtered_df = df[df['Aircraft Reg #'].astype(str).str.startswith('4682')]


import openpyxl
import pandas as pd

excel_file_path = r'C:\Users\vnathani\Desktop\Material_Request_Details_Analysis\MESAZ-145 Parts Needs.xlsm'

wb = openpyxl.load_workbook(excel_file_path, read_only=True)

worksheet_name = '4682'
ws = wb[worksheet_name]

headers = [cell.value.strip() if cell.value else None for cell in ws[2]]

acsn_index = headers.index('ACSN:')
received_index = headers.index('Received')

data_rows = []

for row in ws.iter_rows(min_row=3, values_only=True):
    
    if str(row[acsn_index]) == '4682' and row[acsn_index] is not None:
        data = row[acsn_index:received_index+1]
        data_rows.append(data)

filtered_master = pd.DataFrame(data_rows, columns=headers[acsn_index:received_index+1])

import pandas as pd

for index, row in filtered_df.iterrows():
    match = filtered_master[
        (filtered_master['Material Request'] == row['Material Request #']) &
        (filtered_master['MR Part #'] == row['MR Part #'])
    ]

    if not match.empty:
        matched_index = match.index[0] 
        filtered_master.at[matched_index, 'MR Created'] = row['MR Created Date']
        filtered_master.at[matched_index, 'MR Need'] = row['MR Need Date']
        filtered_master.at[matched_index, 'Part Description'] = row['Part Description']
        filtered_master.at[matched_index, 'MR Qty.'] = row['MR Qty.']
        if row['Issued Qty'] == filtered_master.at[matched_index, 'Issued Qty']:
            pass
        else:
            temp = filtered_master.at[matched_index, 'Issued Qty'] - row['Issued Qty']
            if temp > 0:
                pass
            else:
                filtered_master.at[matched_index, 'Issued Qty'] = row['Issued Qty']
                
        filtered_master.at[matched_index, 'PO#'] = row['PO #']
        filtered_master.at[matched_index, 'STI#'] = row['STI #']
        filtered_master.at[matched_index, 'Vendor Name'] = row['Vendor Name']
        filtered_master.at[matched_index, 'Requested By'] = row['Requested By']
        filtered_master.at[matched_index, 'Task #'] = row['Task#']


In [11]:
last_row_values = filtered_master.loc[filtered_master.index[-1], ['Material Request', 'MR Part #']]
last_row_values

NameError: name 'filtered_master' is not defined

In [12]:
import os
import pandas as pd

# Your initial code to find the row number
try:
    row_number = filtered_df[(filtered_df['Material Request #'] == last_row_values['Material Request']) & (filtered_df['MR Part #'] == last_row_values['MR Part #'])].index[0]
    
    # If there is no error, execute this block
    extended_rows = filtered_df.loc[row_number+1:]
    extended_rows.columns = filtered_master.columns
    filtered_master = pd.concat([filtered_master, extended_rows], ignore_index=True)
    
    # Define file path
    file_path = r'C:\Users\vnathani\airmethods.com\145 Repair Station - 145 Mesa Misc Files\Dashboard\4682_updated.xlsx'

    # Check if the file already exists and remove it
    if os.path.exists(file_path):
        os.remove(file_path)  

    # Save the new Excel file
    filtered_master.to_excel(file_path, index=False)

except IndexError:
    # If the error occurs, execute this block
    file_path = r'C:\Users\vnathani\airmethods.com\145 Repair Station - 145 Mesa Misc Files\Dashboard\4682_updated.xlsx'

    # Check if the file already exists and remove it
    if os.path.exists(file_path):
        os.remove(file_path)  

    # Save the new Excel file
    filtered_master.to_excel(file_path, index=False)


NameError: name 'filtered_df' is not defined